# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> The variables to use are going to be the land use, multi use, parcel size, building size, property value, and agricultural. The reason for this is because the use of the land will tell us what the land is used for. Which could impact pollutants. If the land has multi use then it will have more building and more building with larger sizes means more pollutants. If the land is used for agricultural purposes then the water in the area could have pollutants in the runoff.</font>

In [25]:
(parquet_variables := ([
 'Year',
 'LAKE_NAME',
 'average_secchi_depth',
 'average_total_phosphorus'])
)

['Year', 'LAKE_NAME', 'average_secchi_depth', 'average_total_phosphorus']

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> The numerical variables will be parcel size, number of units, property values, distance to lakes, and year built. The categorical variabes will be land use, house style, amenities, agricultural use. For land use compute residential, agricultural, and commercial uses. Also compute number of parcels with multi use. For house style get counts of single family, multi family, and number of apartments and houses. For amenities get counts of basements, garages, heating, and cooling. For agricultural use get proportion of acres.</font>

### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to 
1. only include parcels within 1600 feet of the lakes we are studying, and 
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [26]:
# your query here

import polars as pl

(parcels := pl.scan_delta("./data/parcel.delta").filter(pl.col("distance_category") != "over 1600 meters"))

In [27]:
(xref := pl.scan_csv('./data/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt',separator='\t',infer_schema=False))

In [28]:
(join_xref_parcels := (xref
                       .join(parcels, on = ['centroid_long', 'centroid_lat'], how = 'inner')
                       .cast({'Year':pl.Int32})
                      )
)

In [29]:
(parquet := pl.scan_parquet('./data/water_quality_by_year.parquet')).collect()

DNR_ID_Site_Number,Year,LAKE_NAME,average_secchi_depth,average_total_phosphorus
str,i32,str,f64,f64
"""82013700-01""",2012,"""Fish Lake""",1.506333,0.108897
"""82009700-01""",2009,"""La Lake""",0.9625,0.133875
"""10009500-01""",2009,"""Swede Lake""",0.410714,0.387929
"""82010300-01""",2006,"""Olson Lake""",2.926,0.0464
"""82009400-01""",2006,"""Colby Lake""",0.642857,0.254143
…,…,…,…,…
"""82010400-01""",2006,"""Jane Lake""",5.122222,0.014
"""82009002-01""",2004,"""Wilmes Lake""",1.408333,0.084917
"""27003501-01""",2011,"""Sweeney Lake""",1.166667,0.041389


In [30]:
(join_parquet := (parquet
    .join(join_xref_parcels, left_on =['DNR_ID_Site_Number', 'Year'], right_on =['Monit_MAP_CODE1', 'Year'], how= 'inner')))

In [31]:
(parquet_lazy := parquet.lazy())

In [32]:
join_xref_parcels_unique_lazy = join_xref_parcels.lazy().unique(subset=['Year', 'Monit_MAP_CODE1'])

In [33]:
(joined_lazy := parquet_lazy.join(
    join_xref_parcels_unique_lazy,
    left_on=['Year', 'DNR_ID_Site_Number'],
    right_on=['Year', 'Monit_MAP_CODE1'],
    how='left'
)
)

In [34]:
joined_lazy.sink_parquet('./data/joined_table.parquet')

In [35]:
(table_all_variables := (pl.read_parquet('./data/joined_table.parquet')))

DNR_ID_Site_Number,Year,LAKE_NAME,average_secchi_depth,average_total_phosphorus,Parcel_PIN,Monit_SITE_CODE,Monit_LAKE_SITE,Distance_Parcel_Monitoring_Site_meters,Lake_Hydroid,Distance_Parcel_Lake_meters,centroid_long,centroid_lat,Parcel_pkey,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,…,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,ZIP,ZIP4,Distance_Parcel_Lake_meters_right,Monit_MAP_CODE1_right,distance_category
str,i32,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str
"""19002500-01""",2004,"""Keller Lake""",1.52,0.039733,null,"""19002500""","""1""","""787.02123559770916""","""110517277034""","""507.08969589819884""","""-93.25938""","""44.73126""","""1106287""","""0.0""","""0.29""",null,null,"""N""",null,"""1101""",null,"""BURNSVILLE""","""BURNSVILLE""",null,"""037""","""S.FAM.RES""","""178700.0""","""55900.0""","""234600.0""","""2252.0""",null,null,"""N""",null,"""Y""","""SPLIT LEVL""",…,"""1.0""","""037-027125001002""",null,null,null,"""1995-08-01""","""145000.0""","""196""","""0.0""","""LAVON LANE""",null,null,"""1158.69718406""","""135.639171617""",null,null,null,"""1981.0""","""N""",null,"""2106.0""",null,"""RESIDENTIAL""",null,null,null,"""BLACK DOG""",null,null,null,null,"""1985.0""","""55337""",null,507.089696,"""19002500-01""","""between_501_and_1600m"""
"""82013700-01""",2013,"""Fish Lake""",1.131923,0.061423,null,"""82013700""","""1""","""2129.765233932219""","""110377674964""","""1939.8661665093232""","""-92.99407""","""45.13783""","""1403446""","""0.0""","""0.1""",null,null,"""N""","""Y""","""5552""",null,"""CITY OF HUGO""","""HUGO""","""Y""","""163""","""Townhouse""","""139700.0""","""33900.0""","""173600.0""","""1237.0""","""Y""","""660.0000""","""N""","""FA Gas""","""Y""","""1 Story Townhouse""",…,"""0.0""","""163-3203121210007""",null,null,null,"""2001-09-28""","""169550.0""","""ISD624""","""0.0""","""129TH""","""DR""","""N""",null,null,"""5552 129TH DR N""",null,"""HUGO MN 55038""","""1520.0""",null,"""PAULSON RAYMOND L & DIANN C""","""0.0""",null,"""100 Res 1 unit""",null,null,null,"""WS RICE CREEK""",null,null,null,null,"""2001.0""","""55038""",null,1939.866167,"""82013700-01""","""over_1600m"""
"""27071100-01""",2013,"""Westwood Lake""",1.4,0.0575,null,"""27071100""","""1""","""2064.7116186581743""","""110407749035""","""1877.6523427040977""","""-93.36636""","""44.96002""","""1157343""","""0.0""","""0.21""",null,null,"""N""","""Y""","""6806""","""002""","""ST. LOUIS PARK""","""ST. LOUIS PARK""","""N""","""053""",null,"""102100.0""","""68100.0""","""170200.0""","""912.0""","""Y""","""288""","""N""","""Forced Air""","""Y""","""Rambler""",…,"""0.0""","""053-0811721130043""","""CEDARCREST 2ND""",null,null,"""2006-07-01""","""220000.0""","""283""","""0.0""","""23RD ST W""",null,null,null,null,"""6806 23RD ST W""","""ST. LOUIS PARK MN 55426""",null,"""1483.0""","""N""","""KATHERINE M THIEKING""","""2286.0""",null,"""Residential""",null,null,null,"""Minnehaha Creek""",null,null,null,null,"""1955.0""","""55426""",null,1877.652343,"""27071100-01""","""over_1600m"""
"""19002601-01""",2006,"""Marion Lake""",2.016667,0.046467,null,"""19002601""","""1""","""2084.5612726288296""","""110517277036""","""410.58935232127158""","""-93.29715""","""44.66897""","""1068182""","""0.0""","""19.04""",null,null,"""N""",null,null,null,"""LAKEVI

In [36]:
table_all_variables = table_all_variables.with_columns(
    [pl.col(c).cast(pl.Float64, strict=False).alias(c) for c in table_all_variables.columns]
)

In [37]:
table_all_variables

DNR_ID_Site_Number,Year,LAKE_NAME,average_secchi_depth,average_total_phosphorus,Parcel_PIN,Monit_SITE_CODE,Monit_LAKE_SITE,Distance_Parcel_Monitoring_Site_meters,Lake_Hydroid,Distance_Parcel_Lake_meters,centroid_long,centroid_lat,Parcel_pkey,ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,…,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,ZIP,ZIP4,Distance_Parcel_Lake_meters_right,Monit_MAP_CODE1_right,distance_category
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,2004.0,null,1.52,0.039733,null,1.90025e7,1.0,787.021236,1.1052e11,507.089696,-93.25938,44.73126,1.106287e6,0.0,0.29,null,null,null,null,1101.0,null,null,null,null,37.0,null,178700.0,55900.0,234600.0,2252.0,null,null,null,null,null,null,…,1.0,null,null,null,null,null,145000.0,196.0,0.0,null,null,null,1158.697184,135.639172,null,null,null,1981.0,null,null,2106.0,null,null,null,null,null,null,null,null,null,null,1985.0,55337.0,null,507.089696,null,null
null,2013.0,null,1.131923,0.061423,null,8.20137e7,1.0,2129.765234,1.1038e11,1939.866167,-92.99407,45.13783,1.403446e6,0.0,0.1,null,null,null,null,5552.0,null,null,null,null,163.0,null,139700.0,33900.0,173600.0,1237.0,null,660.0,null,null,null,null,…,0.0,null,null,null,null,null,169550.0,null,0.0,null,null,null,null,null,null,null,null,1520.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,2001.0,55038.0,null,1939.866167,null,null
null,2013.0,null,1.4,0.0575,null,2.70711e7,1.0,2064.711619,1.1041e11,1877.652343,-93.36636,44.96002,1.157343e6,0.0,0.21,null,null,null,null,6806.0,2.0,null,null,null,53.0,null,102100.0,68100.0,170200.0,912.0,null,288.0,null,null,null,null,…,0.0,null,null,null,null,null,220000.0,283.0,0.0,null,null,null,null,null,null,null,null,1483.0,null,null,2286.0,null,null,null,null,null,null,null,null,null,null,1955.0,55426.0,null,1877.652343,null,null
null,2006.0,null,2.016667,0.046467,null,1.9002601e7,1.0,2084.561273,1.1052e11,410.589352,-93.29715,44.66897,1.068182e6,0.0,19.04,null,null,null,null,null,null,null,null,null,37.0,null,0.0,113800.0,113800.0,0.0,null,null,null,null,null,null,…,1.0,null,null,null,null,null,0.0,194.0,0.0,null,null,null,76973.433662,1595.311879,null,null,null,0.0,null,null,0.0,null,null,null,null,null,null,null,null,null,null,0.0,null,null,410.589352,null,null
null,2014.0,null,1.491667,0.094667,null,1.00052e7,1.0,2593.811836,1.1022e11,1757.174905,-93.77682,44.83454,2.008119e6,0.0,0.14,null,null,null,null,524.0,2.0,null,null,null,19.0,null,0.0,50200.0,50200.0,0.0,null,null,null,null,null,null,…,1.0,null,null,null,null,null,0.0,110.0,0.0,null,null,null,null,null,null,null,null,628.0,null,null,920.0,null,null,null,null,null,null,null,null,null,null,0.0,55387.0,5387.0,1757.174905,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,2014.0,null,1.456364,0.034591,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,2005.0,null,1.915929,0.034714,null,null,null,null,null,n

In [38]:
#table_all_variables = table_all_variables.with_columns([
#    pl.col("DNR_ID_Site_Number").cast(pl.String),
#    pl.col("LakeName").cast(pl.Int32),
#    pl.col("Year").cast(pl.String),
#])

In [39]:
table_all_variables = table_all_variables.select(
    pl.col('Year', 'LAKE_NAME', 'average_secchi_depth', 'average_total_phosphorus', 'EMV_TOTAL',
           'SALE_VALUE', 'TOTAL_TAX', 'GARAGESQFT', 'FIN_SQ_FT', 'BASEMENT', 'GARAGE', 'TAX_EXEMPT', 'COOLING', 'HEATING'))

In [40]:
table_all_variables = table_all_variables.with_columns([
    pl.col("Year").cast(pl.Int32),
    pl.col('EMV_TOTAL').cast(pl.Float64),
    pl.col('SALE_VALUE').cast(pl.Float64),
    pl.col('TOTAL_TAX').cast(pl.Float64),
    pl.col('GARAGESQFT').cast(pl.Float64),
    pl.col('FIN_SQ_FT').cast(pl.Float64),
    pl.col("BASEMENT").cast(pl.String),
    pl.col("GARAGE").cast(pl.String),
    pl.col("TAX_EXEMPT").cast(pl.String),
    pl.col("COOLING").cast(pl.String),
    pl.col("HEATING").cast(pl.String),
])

In [41]:
table_all_variables

Year,LAKE_NAME,average_secchi_depth,average_total_phosphorus,EMV_TOTAL,SALE_VALUE,TOTAL_TAX,GARAGESQFT,FIN_SQ_FT,BASEMENT,GARAGE,TAX_EXEMPT,COOLING,HEATING
i32,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str
2004,null,1.52,0.039733,234600.0,145000.0,2106.0,null,2252.0,null,null,null,null,null
2013,null,1.131923,0.061423,173600.0,169550.0,0.0,660.0,1237.0,null,null,null,null,null
2013,null,1.4,0.0575,170200.0,220000.0,2286.0,288.0,912.0,null,null,null,null,null
2006,null,2.016667,0.046467,113800.0,0.0,0.0,null,0.0,null,null,null,null,null
2014,null,1.491667,0.094667,50200.0,0.0,920.0,null,0.0,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2014,null,1.456364,0.034591,null,null,null,null,null,null,null,null,null,null
2005,null,1.915929,0.034714,null,null,null,null,null,null,null,null,null,null
2007,null,1.7,0.0202,null,null,null,null,null,null,null,null,null,null


## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.** 

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

In [42]:
size_vars = ["FIN_SQ_FT", "GARAGESQFT"]
value_vars = ["EMV_TOTAL", "SALE_VALUE", "TOTAL_TAX"]

all_vars = size_vars + value_vars

numerical_summary = (
    table_all_variables
    .group_by(["LAKE_NAME", "Year"])
    .agg([
        *[
            pl.col(v).mean().alias(f"{v}_mean") 
            for v in all_vars
        ],
        *[
            pl.col(v).std().alias(f"{v}_std") 
            for v in all_vars
        ],
    ])
)

In [43]:
numerical_summary

LAKE_NAME,Year,FIN_SQ_FT_mean,GARAGESQFT_mean,EMV_TOTAL_mean,SALE_VALUE_mean,TOTAL_TAX_mean,FIN_SQ_FT_std,GARAGESQFT_std,EMV_TOTAL_std,SALE_VALUE_std,TOTAL_TAX_std
f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
null,2006,1457.5625,580.0,450922.916667,124540.458333,4581.5625,1448.209252,197.989899,766644.012436,152577.808807,10948.491181
null,2014,1457.5625,623.653846,373791.666667,172999.979167,3558.270833,2404.4093,495.39616,573849.900248,514004.494721,11975.880993
null,2007,693.833333,731.5,362650.0,120266.458333,1414.395833,1128.142388,412.243253,402382.296256,194341.247769,2080.281709
null,2010,1285.229167,620.333333,434925.0,316164.0625,6338.541667,1036.672259,209.214778,970708.061269,898989.313968,34293.374694
null,2005,433.354167,672.0,250656.25,98388.458333,2251.895833,861.934501,null,274568.818569,218292.922097,4472.251869
…,…,…,…,…,…,…,…,…,…,…,…
null,2004,543.375,644.0,271796.166667,71800.1875,2249.416667,915.087105,39.59798,196862.211428,127978.977114,2169.97175
null,2013,1381.104167,527.96875,334166.666667,194283.166667,1771.5,1613.819182,396.583025,513667.722855,554460.433424,3531.911001
null,2009,1245.229167,489.666667,336102.083333,153662.9375,155494.541667,1099.639385,156.840896,352701.981058,172374.243706,326570.423818


In [44]:
numerical_summary.write_csv("./data/parcel_numerical_summaries.csv")

## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* DWELL_TYPE: Dwelling Type 
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [49]:
# Your code here

categorical_variables = table_all_variables.select([
    "LAKE_NAME",
    "Year",
    "GARAGE",
    "HEATING",
    "COOLING",
])

categorical_variables = categorical_variables.with_columns([
    pl.col("GARAGE")
        .cast(pl.Utf8)
        .str.to_uppercase()
        .str.strip_chars()
        .replace({"YES": "Y", "NO": "N"})
        .alias("GARAGE"),

    pl.col("HEATING")
        .cast(pl.Utf8)
        .str.to_uppercase()
        .str.strip_chars()
        .alias("HEATING"),

    pl.col("COOLING")
        .cast(pl.Utf8)
        .str.to_uppercase()
        .str.strip_chars()
        .alias("COOLING"),
])

categorical_variables = categorical_variables.with_columns(
    indicator = pl.lit(1)
)


heating_summary = (
    categorical_variables.group_by(["LAKE_NAME", "Year", "HEATING"])
      .agg(pl.sum("indicator").alias("count"))
      .pivot(
          values="count",
          index=["LAKE_NAME", "Year"],
          on="HEATING"
      )
      .fill_null(0)
)

cooling_summary = (
    categorical_variables.group_by(["LAKE_NAME", "Year", "COOLING"])
      .agg(pl.sum("indicator").alias("count"))
      .pivot(
          values="count",
          index=["LAKE_NAME", "Year"],
          on="COOLING"
      )
      .fill_null(0)
)

heating_summary = heating_summary.rename(lambda c: f"HEATING_{c}" if c not in ["LAKE_NAME", "Year"] else c)
cooling_summary = cooling_summary.rename(lambda c: f"COOLING_{c}" if c not in ["LAKE_NAME", "Year"] else c)


categorical_summary = (
    heating_summary
    .join(cooling_summary, on=["LAKE_NAME", "Year"], how="left")
)

categorical_summary.write_csv("./data/parcel_categorical_summaries.csv")

## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [46]:
# Your code here.

(all_summaries := (table_all_variables.select(parquet_variables)
                   .join(numerical_summary, on=["LAKE_NAME", "Year"], how="left")
                   .join(categorical_summary, on=["LAKE_NAME", "Year"], how="left")
)
)

Year,LAKE_NAME,average_secchi_depth,average_total_phosphorus,FIN_SQ_FT_mean,GARAGESQFT_mean,EMV_TOTAL_mean,SALE_VALUE_mean,TOTAL_TAX_mean,FIN_SQ_FT_std,GARAGESQFT_std,EMV_TOTAL_std,SALE_VALUE_std,TOTAL_TAX_std,HEATING_null,COOLING_null
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32
2004,null,1.52,0.039733,null,null,null,null,null,null,null,null,null,null,null,null
2013,null,1.131923,0.061423,null,null,null,null,null,null,null,null,null,null,null,null
2013,null,1.4,0.0575,null,null,null,null,null,null,null,null,null,null,null,null
2006,null,2.016667,0.046467,null,null,null,null,null,null,null,null,null,null,null,null
2014,null,1.491667,0.094667,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2014,null,1.456364,0.034591,null,null,null,null,null,null,null,null,null,null,null,null
2005,null,1.915929,0.034714,null,null,null,null,null,null,null,null,null,null,null,null
2007,null,1.7,0.0202,null,null,null,null,null,null,null,null,null,null,null,null


In [47]:
all_summaries.write_csv("./data/water_quality_and_parcel_summaries_2004_to_2015MINE.csv")

## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.

In [48]:
# Your code here.